In [1]:
#Import Dependencies
import psycopg2
import sqlalchemy
import pandas as pd 
from sqlalchemy import create_engine
from config import password 

In [2]:
# Read in CSVs
year = [2018, 2019, 2020, 2021]
df_names =[]
df_list = []


for y in year:
    filename = f"weekly{y}_playerid.csv"
    filepath = f"../../DataBase/Resources/weekly_stats/"
    df = f"{y}_weekly_df"
    df_names.append(df)
    read_df = pd.read_csv(f"{filepath}{filename}")
    df_list.append(read_df)

      

In [3]:
#Check df names
df_names

['2018_weekly_df', '2019_weekly_df', '2020_weekly_df', '2021_weekly_df']

In [4]:
#check data list 
len(df_list)

4

In [5]:
#Add year to each dataframe
for i in range(0,len(df_list)):
    yr = df_names[i][:4]
    df_list[i]["Year"] = yr

In [6]:
#Check year was added
for i in range(0,len(df_list)):
    print(df_list[i].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Rank                  2500 non-null   int64  
 1   PlayerID              2500 non-null   int64  
 2   Name                  2500 non-null   object 
 3   Team                  2500 non-null   object 
 4   Position              2500 non-null   object 
 5   Week                  2500 non-null   int64  
 6   Opponent              2500 non-null   object 
 7   PassingYards          2500 non-null   int64  
 8   PassingTouchdowns     2500 non-null   int64  
 9   PassingInterceptions  2500 non-null   int64  
 10  RushingYards          2500 non-null   int64  
 11  RushingTouchdowns     2500 non-null   int64  
 12  Receptions            2500 non-null   int64  
 13  ReceivingYards        2500 non-null   int64  
 14  ReceivingTouchdowns   2500 non-null   int64  
 15  FumblesLost          

In [7]:
#Create empty DF to concatenate position
weekly_stats_df = pd.DataFrame()

In [8]:
#Use coniditionals to concat dataframs
for i in range(0,len(df_list)):
    weekly_stats_df = pd.concat([df_list[i], weekly_stats_df], ignore_index=True)
    print(f"index {i} was added to df")


index 0 was added to df
index 1 was added to df
index 2 was added to df
index 3 was added to df


In [9]:
#Quick Check on QBs
weekly_stats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Rank                  10000 non-null  int64  
 1   PlayerID              10000 non-null  int64  
 2   Name                  10000 non-null  object 
 3   Team                  10000 non-null  object 
 4   Position              10000 non-null  object 
 5   Week                  10000 non-null  int64  
 6   Opponent              10000 non-null  object 
 7   PassingYards          10000 non-null  int64  
 8   PassingTouchdowns     10000 non-null  int64  
 9   PassingInterceptions  10000 non-null  int64  
 10  RushingYards          10000 non-null  int64  
 11  RushingTouchdowns     10000 non-null  int64  
 12  Receptions            10000 non-null  int64  
 13  ReceivingYards        10000 non-null  int64  
 14  ReceivingTouchdowns   10000 non-null  int64  
 15  FumblesLost         

In [10]:
#format columns to lower case for db
weekly_stats_df.columns = weekly_stats_df.columns.str.lower()

Start SQL Imports

In [11]:
#Create connection to AWS database
ENDPOINT="fantasyfootball.ckswyhqaro8q.us-east-2.rds.amazonaws.com"
PORT="5432"
USR="DreamTeam"
PSSWD=password
DBNAME="postgres"

#conn_string = "host='localhost' dbname='my_database' user='postgres' password='secret'
con= create_engine("postgresql://{}:{}@{}:{}/{}".format(USR, PSSWD, ENDPOINT, PORT, DBNAME))

In [12]:
#Push table to AWS 
weekly_stats_df.to_sql("weekly_stats", con, if_exists="replace")